#### 1. Download the college dataset:

In [1]:
%env BASE_DIR=/home/jupyter/snippets

env: BASE_DIR=/home/jupyter/snippets


In [2]:
!gsutil cp gs://cs327e-open-access/fs_college.zip $BASE_DIR

Copying gs://cs327e-open-access/fs_college.zip...
/ [1 files][  2.0 KiB/  2.0 KiB]                                                
Operation completed over 1 objects/2.0 KiB.                                      


In [3]:
!unzip $BASE_DIR/fs_college.zip

Archive:  /home/jupyter/snippets/fs_college.zip
  inflating: fs_college/class.csv    
  inflating: fs_college/instructor.csv  
  inflating: fs_college/student.csv  
  inflating: fs_college/takes.csv    
  inflating: fs_college/teaches.csv  


#### 2. Create and populate Firestore database

In [4]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('fs_college/class.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

for row in rows: 
    record = {}
    record['cno'] = row[0]
    record['cname'] = row[1]
    record['credits'] = row[2]
    
    class_ref = db.collection('class').document(row[0])
    batch.set(class_ref, record)
    
batch.commit()

[update_time {
   seconds: 1697229472
   nanos: 956320000
 },
 update_time {
   seconds: 1697229472
   nanos: 956320000
 },
 update_time {
   seconds: 1697229472
   nanos: 956320000
 },
 update_time {
   seconds: 1697229472
   nanos: 956320000
 },
 update_time {
   seconds: 1697229472
   nanos: 956320000
 },
 update_time {
   seconds: 1697229472
   nanos: 956320000
 },
 update_time {
   seconds: 1697229472
   nanos: 956320000
 },
 update_time {
   seconds: 1697229472
   nanos: 956320000
 },
 update_time {
   seconds: 1697229472
   nanos: 956320000
 }]

In [5]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('fs_college/student.csv', sep=',', header=0, lineterminator='\n')
student_rows = df.values.tolist()

df = pd.read_csv('fs_college/takes.csv', sep=',', header=0, lineterminator='\n')
takes_rows = df.values.tolist()

for student_row in student_rows:
    student_record = {}
    student_record['sid'] = student_row[0]
    student_record['fname'] = student_row[1]
    student_record['lname'] = student_row[2]
    student_record['dob'] = student_row[3]
    student_record['status'] = student_row[4]
    
    student_ref = db.collection('student').document(student_row[0])
    
    batch.set(student_ref, student_record)
    
    for takes_row in takes_rows:
        if student_row[0] == takes_row[0]:
            class_record = {}
            class_record['cno'] = takes_row[1]
            class_record['cname'] = takes_row[2]
            class_record['credits'] = takes_row[3]
            class_record['grade'] = takes_row[4]
            
            classes_ref = student_ref.collection('classes').document(takes_row[1])
        
            batch.set(classes_ref, class_record)
    
    batch.commit()

In [6]:
batch = db.batch()

df = pd.read_csv('fs_college/instructor.csv', sep=',', header=0, lineterminator='\n')
instructor_rows = df.values.tolist()

df = pd.read_csv('fs_college/teaches.csv', sep=',', header=0, lineterminator='\n')
teaches_rows = df.values.tolist()

for instructor_row in instructor_rows:
    
    instructor_record = {}
    instructor_record['tid'] = instructor_row[0]
    instructor_record['instructor_name'] = instructor_row[1]
    instructor_record['dept'] = instructor_row[2]
    
    instructor_ref = db.collection('instructor').document(instructor_row[0])
    
    batch.set(instructor_ref, instructor_record)
    
    for teaches_row in teaches_rows:
        
        if instructor_row[0] == teaches_row[0]:
            
            class_record = {}
            class_record['cno'] = teaches_row[1]
            class_record['cname'] = teaches_row[2]
            class_record['credits'] = teaches_row[3]
            
            classes_ref = instructor_ref.collection('classes').document(teaches_row[1])
        
            batch.set(classes_ref, class_record)
    
    batch.commit()

#### 3. Run some queries

In [7]:
student_ref = db.collection('student').document('paulg')
result = student_ref.get()

if result.exists:
    print(f'{result.id} => {result.to_dict()}')
else:
    print('No such student')

paulg => {'lname': 'Gore', 'status': 'CUR', 'fname': 'Paul', 'sid': 'paulg', 'dob': '2000-09-17'}


In [9]:
from google.cloud.firestore_v1.base_query import FieldFilter
student_ref = db.collection('student')
query = student_ref.where(filter=FieldFilter('status', '==', 'CUR'))
results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

aprilz => {'status': 'CUR', 'sid': 'aprilz', 'dob': '2000-10-01', 'fname': 'April', 'lname': 'Lopez'}
bzen26 => {'dob': '1998-04-22', 'fname': 'Biswa', 'lname': 'Zen', 'status': 'CUR', 'sid': 'bzen26'}
jc => {'lname': 'Cowe', 'status': 'CUR', 'fname': 'James', 'dob': '2000-04-22', 'sid': 'jc'}
jerryh => {'status': 'CUR', 'fname': 'Jerry', 'sid': 'jerryh', 'lname': 'Hargrove', 'dob': '1999-01-03'}
kev18 => {'status': 'CUR', 'lname': 'Lin', 'sid': 'kev18', 'fname': 'Kevin', 'dob': '1999-10-05'}
paulg => {'sid': 'paulg', 'fname': 'Paul', 'lname': 'Gore', 'dob': '2000-09-17', 'status': 'CUR'}
sudeepa4 => {'sid': 'sudeepa4', 'lname': 'Roy', 'dob': '2001-10-01', 'fname': 'Sudeepa', 'status': 'CUR'}


In [10]:
query = db.collection('instructor').document('mitra').collection('classes')
results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

CS313E => {'credits': 3, 'cname': 'Elements of Software Engineering', 'cno': 'CS313E'}
CS329E => {'cname': 'Elements of Web Programming', 'cno': 'CS329E', 'credits': 3}


### Prompt 1: Get all classes which are worth 3 credits. Return the attributes of those classes. 

In [11]:
from google.cloud.firestore_v1.base_query import FieldFilter
class_ref = db.collection('class')
query = class_ref.where(filter=FieldFilter('credits', '==', 3))
results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

CS303E => {'cname': 'Elements of Computers and Programming', 'credits': 3, 'cno': 'CS303E'}
CS313E => {'cno': 'CS313E', 'credits': 3, 'cname': 'Elements of Software Engineering'}
CS326E => {'cno': 'CS326E', 'cname': 'Elements of Networking', 'credits': 3}
CS327E => {'cno': 'CS327E', 'cname': 'Elements of Databases', 'credits': 3}
CS329E => {'cname': 'Elements of Web Programming', 'credits': 3, 'cno': 'CS329E'}
CS347 => {'cno': 'CS347', 'cname': 'Data Management', 'credits': 3}
M328K => {'cname': 'Intro to Number Theory', 'cno': 'M328K', 'credits': 3}
M358K => {'cno': 'M358K', 'credits': 3, 'cname': 'Applied Statistics'}
M362K => {'cno': 'M362K', 'credits': 3, 'cname': 'Probability I'}


### Prompt 2: What does the following query do? Write a short markdown description below the code

In [12]:
instructor_ref = db.collection('instructor')
results = instructor_ref.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')
    sresults = instructor_ref.document(result.id).collection('classes').stream()
    
    for sresult in sresults:
        print(f'{sresult.id} => {sresult.to_dict()}')

bulko => {'dept': 'Computer Science', 'tid': 'bulko', 'instructor_name': 'Bill Bulko'}
CS303E => {'credits': 3, 'cno': 'CS303E', 'cname': 'Elements of Computers and Programming'}
cannata => {'tid': 'cannata', 'dept': 'Computer Science', 'instructor_name': 'Phil Cannata'}
CS326E => {'cno': 'CS326E', 'cname': 'Elements of Networking', 'credits': 3}
downing => {'instructor_name': 'Glenn Downing', 'tid': 'downing', 'dept': 'Computer Science'}
fares => {'tid': 'fares', 'instructor_name': 'Fares Fraij', 'dept': 'Computer Science'}
koch => {'tid': 'koch', 'dept': 'Mathematics', 'instructor_name': 'Hans Koch'}
M328K => {'cno': 'M328K', 'credits': 3, 'cname': 'Intro to Number Theory'}
mitra => {'tid': 'mitra', 'instructor_name': 'Shyamal Mitra', 'dept': 'Computer Science'}
CS313E => {'credits': 3, 'cno': 'CS313E', 'cname': 'Elements of Software Engineering'}
CS329E => {'cname': 'Elements of Web Programming', 'cno': 'CS329E', 'credits': 3}
mueller => {'dept': 'Mathematics', 'tid': 'mueller', 'in